# Prepare prerequisites for the training and deployment steps.



in the first cell of the Jupyter Notebook, use pip install to upgrade sagemaker[local]

In [1]:
!pip install 'sagemaker[local]' --upgrade

Specify the bucket name where the training_data.csv file is stored. Use the bucket name we created in the Preparing the Amazon S3 bucket and the training dataset for the linear regression experiment recipe of Chapter 1, Getting Started with Machine Learning Using Amazon SageMaker:

In [2]:
s3_bucket = "ml-sagemaker-cookbook-bucket"
prefix = "chapter01"

Set the variable values for training_s3_input_location and training_s3_output_location

In [3]:
training_s3_input_location = f"s3://{s3_bucket}/{prefix}/input/training_data.csv"
training_s3_output_location = f"s3://{s3_bucket}/{prefix}/output/custom/"

Import the SageMaker Python SDK and check its version

In [4]:
import sagemaker

sagemaker.__version__

'2.109.0'

Set the value of the container image.

In [12]:
container = "469889560361.dkr.ecr.eu-west-1.amazonaws.com/chap02_python:1"

Import a few prerequisites such as role and session
The LocalSession class allows us to use local mode in the training and deployment steps

In [6]:
import boto3
from sagemaker import get_execution_role

try:
    role = get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20210615T144455')['Role']['Arn']

from sagemaker.local import LocalSession

session = LocalSession()
session.config = {'local': {'local_code': True}}

Couldn't call 'get_role' to get Role ARN from role name letlhogile@gometroapp.com to get Role path.


Initialize the TrainingInput object for the train data channel

In [7]:
from sagemaker.inputs import TrainingInput

train = TrainingInput(training_s3_input_location, content_type="text/csv")

# Train Model

Initialize Estimator and use container, role, session, and training_s3_output_location as the parameter values when initializing the Estimator object

In [13]:
estimator = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type='local',
    output_path=training_s3_output_location,
    sagemaker_session=session)

Set a few dummy hyperparameters by using the set_hyperparameters() function. Behind the scenes, these values will passed to the hyperparameters.json file inside the /opt/ml/input/config directory, which will be loaded and used by the train script when we run the fit() function later

In [14]:
estimator.set_hyperparameters(a=1, b=2, c=3)

Start the training job using fit()

In [15]:
# !docker login -u AWS -p $password 469889560361.dkr.ecr.eu-west-1.amazonaws.com
!aws ecr get-login-password --region eu-west-1 | docker login --username AWS --password-stdin 469889560361.dkr.ecr.eu-west-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/letlhogile/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [16]:
estimator.fit({'train': train})

Creating 58r34qk9hh-algo-1-01s4r ... 
Creating 58r34qk9hh-algo-1-01s4r ... done
Attaching to 58r34qk9hh-algo-1-01s4r
58r34qk9hh-algo-1-01s4r | [inspect_hyperparameters]
58r34qk9hh-algo-1-01s4r | /opt/ml/input/config/hyperparameters.json
58r34qk9hh-algo-1-01s4r | {'a': '1', 'b': '2', 'c': '3'}
58r34qk9hh-algo-1-01s4r | [inspect_input]
58r34qk9hh-algo-1-01s4r | /opt/ml/input/config/inputdataconfig.json
58r34qk9hh-algo-1-01s4r | {'train': {'ContentType': 'text/csv', 'TrainingInputMode': 'File'}}
58r34qk9hh-algo-1-01s4r | [get_input_data_dir]
58r34qk9hh-algo-1-01s4r | [load_training_data]
58r34qk9hh-algo-1-01s4r | [list_dir_contents]
58r34qk9hh-algo-1-01s4r | ['training_data.csv']
58r34qk9hh-algo-1-01s4r | /opt/ml/input/data/train/training_data.csv
58r34qk9hh-algo-1-01s4r |        0   1
58r34qk9hh-algo-1-01s4r | 0   1020  15
58r34qk9hh-algo-1-01s4r | 1   1390  49
58r34qk9hh-algo-1-01s4r | 2   1590  94
58r34qk9hh-algo-1-01s4r | 3   1290  38
58r34qk9hh-algo-1-01s4r | 4   1750  93
58r34qk9hh-

Use the deploy() function to deploy the inference endpoint:

In [17]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type='local',
    endpoint_name="custom-local-py-endpoint")

Attaching to pq9brxh0vp-algo-1-0orlm
pq9brxh0vp-algo-1-0orlm |  * Serving Flask app "serve" (lazy loading)
pq9brxh0vp-algo-1-0orlm |  * Environment: production
pq9brxh0vp-algo-1-0orlm |    WARNING: This is a development server. Do not use it in a production deployment.
pq9brxh0vp-algo-1-0orlm |    Use a production WSGI server instead.
pq9brxh0vp-algo-1-0orlm |  * Debug mode: off
pq9brxh0vp-algo-1-0orlm |  * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
pq9brxh0vp-algo-1-0orlm | 172.18.0.1 - - [22/Sep/2022 11:47:38] "GET /ping HTTP/1.1" 200 -
!

In [18]:
predictor.predict("1")

pq9brxh0vp-algo-1-0orlm | /opt/ml/model/model
pq9brxh0vp-algo-1-0orlm | 172.18.0.1 - - [22/Sep/2022 11:48:10] "POST /invocations HTTP/1.1" 200 -


b'907.8777829046026'

In [19]:
predictor.delete_endpoint()

Gracefully stopping... (press Ctrl+C again to force)
